In [1]:
import pandas as pd
import numpy as np
import os

## Concatenate several datasets

In [95]:
def get_dataset(directory):
    list_dir = [file for file in os.listdir(directory) if file.endswith('.csv')]
    dfs = []
    for file in list_dir:
        file_path = os.path.join(directory, file)
        df = pd.read_csv(file_path)
        df.dropna(how='all', inplace=True, ignore_index=True)
        if '_metro' in file:
            df['metro'] = 1
        else: 
            df['metro'] = 0
        dfs.append(df)

    concatenated_df = pd.concat(dfs, ignore_index=True)

    return concatenated_df    

In [96]:
flats = get_dataset('datasets')

In [97]:
flats.to_csv('flats.csv')

In [98]:
flats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25769 entries, 0 to 25768
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   price   25769 non-null  object
 1   dist1   25769 non-null  object
 2   dist2   25608 non-null  object
 3   dist3   24226 non-null  object
 4   rooms   25769 non-null  object
 5   square  25769 non-null  object
 6   floor   25769 non-null  object
 7   type1   22015 non-null  object
 8   type2   20235 non-null  object
 9   type3   20647 non-null  object
 10  metro   25769 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 2.2+ MB


## Removing duplicates

In [99]:
flats.drop_duplicates(subset=['price', 'dist1', 'dist2', 'dist3', 'rooms', 'square', 'floor', 'type1', 'type2', 'type3'],
                   inplace=True, ignore_index=True)

In [100]:
flats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15547 entries, 0 to 15546
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   price   15547 non-null  object
 1   dist1   15547 non-null  object
 2   dist2   15453 non-null  object
 3   dist3   14504 non-null  object
 4   rooms   15547 non-null  object
 5   square  15547 non-null  object
 6   floor   15547 non-null  object
 7   type1   13101 non-null  object
 8   type2   11979 non-null  object
 9   type3   12855 non-null  object
 10  metro   15547 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 1.3+ MB


## Fix column values

Columns dist1, dist2, and dist3 represent residential complex, microdistrict, and districts respectively. But some flat advertisements don't have all three recordings so that district can be recorded in the dist2 or dist1 column.

In [101]:
flats.dist3.value_counts()

dist3
Київ              7518
Печерський        2357
Голосіївський     1165
Дарницький         934
Шевченківський     768
Солом'янський      464
Дніпровський       449
Оболонський        303
Подільський        291
Святошинський      193
Деснянський         62
Name: count, dtype: int64

In [102]:
flats.dist3 = flats.dist3.apply(lambda x : np.nan if x=='Київ' else x)

In [103]:
districts = ['Печерський', 'Деснянський', 'Шевченківський', 'Дніпровський', 'Голосіївський',
             "Солом'янський", 'Святошинський', 'Оболонський', 'Дарницький', 'Подільський']

In [104]:
for i in range(0, len(flats)):
    if flats.dist1[i] in districts:
        flats.loc[i, 'dist3'] = flats.dist1[i]
        flats.loc[i, 'dist1'] = np.nan
    elif flats.dist2[i] in districts:
        flats.loc[i, 'dist3'] = flats.dist2[i]
        flats.loc[i, 'dist2'] = np.nan
    

The same situation with columns type1, type2 type3. Columns type1 and type2 have to contain information about the type of the building and column type3 - the year. But sometimes the year is recorded in type1 or type2 columns.

In [105]:
import re
pattern = r'\d{4}'
for i in range(0, len(flats)):
    if re.match(pattern, str(flats.type1[i])):
        flats.loc[i, 'type3'] = flats.type1[i]
        flats.loc[i, 'type1'] = np.nan
    elif re.match(pattern, str(flats.type2[i])):
        flats.loc[i, 'type3'] = flats.type2[i]
        flats.loc[i, 'type2'] = np.nan

In [106]:
flats.head()

,price,dist1,dist2,dist3,rooms,square,floor,type1,type2,type3,metro
0,26 000 грн,ЖК Місце мрії,Шулявка,Солом'янський,2 кімнати,60 / 35 / 15 м²,поверх 14 з 25,спец. проект,монолітно-каркасний,2009.0,1
1,95 750 грн,ЖК Central Park,Нова забудова,Печерський,3 кімнати,131 / 80 / 15 м²,поверх 10 з 25,NaN,NaN,NaN,1
2,57 450 грн,ЖК Новопечерські Липки,Чорна Гора,Печерський,2 кімнати,87 / 41 / 17 м²,поверх 2 з 23,спец. проект,монолітно-каркасний,2013.0,1
3,8 000 грн,Харьківський,NaN,Дарницький,1 кімната,40 / 18 / 9 м²,поверх 3 з 16,серія Т,утеплена панель,1983.0,1
4,15 000 грн,ЖМ Патріотика,Осокорки,Дарницький,1 кімната,38 / 12 / 16 м²,поверх 15 з 25,спец. проект,панельні,2018.0,1


## Fix values type

### Fix price column values

In [107]:
flats.price = flats.price.apply(lambda x:  x.replace(' ', '')) #delete space between numbers
flats.price.replace('[^0-9]', '', regex=True, inplace=True) #delete all not numeric symbols
flats.price = flats.price.astype('int32') 

### Fix rooms column values

In [108]:
df.rooms.unique()

array(['2', nan, '3', '1', '4', '6', '5'], dtype=object)

In [109]:
flats['rooms'].replace('[^0-9]', '', regex=True, inplace=True)
flats['rooms'] = flats.rooms.astype('int32')

### Fix area column values

In [110]:
flats['square'].replace('м²', '', inplace=True, regex=True)

str_split = flats.square.str.split(' / ') # split area values in three separate columns

flats['total_area'] = pd.to_numeric(str_split.str.get(0), errors='coerce') 
flats['living_area'] = pd.to_numeric(str_split.str.get(1), errors='coerce')
flats['kitchen_area']= pd.to_numeric(str_split.str.get(2), errors='coerce')
flats.head()

,price,dist1,dist2,dist3,rooms,square,floor,type1,type2,type3,metro,total_area,living_area,kitchen_area
0,26000,ЖК Місце мрії,Шулявка,Солом'янський,2,60 / 35 / 15,поверх 14 з 25,спец. проект,монолітно-каркасний,2009.0,1,60.0,35.0,15.0
1,95750,ЖК Central Park,Нова забудова,Печерський,3,131 / 80 / 15,поверх 10 з 25,NaN,NaN,NaN,1,131.0,80.0,15.0
2,57450,ЖК Новопечерські Липки,Чорна Гора,Печерський,2,87 / 41 / 17,поверх 2 з 23,спец. проект,монолітно-каркасний,2013.0,1,87.0,41.0,17.0
3,8000,Харьківський,NaN,Дарницький,1,40 / 18 / 9,поверх 3 з 16,серія Т,утеплена панель,1983.0,1,40.0,18.0,9.0
4,15000,ЖМ Патріотика,Осокорки,Дарницький,1,38 / 12 / 16,поверх 15 з 25,спец. проект,панельні,2018.0,1,38.0,12.0,16.0


### Fix floor column values

In [111]:
flats['floor'].replace('поверх', '', regex=True, inplace=True)

str_split = flats.floor.str.split(' з ')

flats['floor'] = pd.to_numeric(str_split.str.get(0), errors='coerce', downcast='integer') 
flats['floor_total'] =  pd.to_numeric(str_split.str.get(1), errors='coerce', downcast='integer')


### Fix year column values

In [113]:
flats.rename(columns= {'type3' :'year'}, inplace = True)
flats.year = pd.to_numeric(flats.year, errors='coerce', downcast='integer') 

### Null values

In [116]:
flats.isna().sum()

price              0
dist1            803
dist2           7518
dist3            334
rooms              0
square             0
floor              2
type1           2635
type2           4060
year            7726
metro              0
total_area         0
living_area     5206
kitchen_area     248
floor_total      152
dtype: int64

In [117]:
flats.dropna(subset = ['dist3', 'floor'], inplace=True, ignore_index=True)

In [118]:
flats.drop(['dist1', 'dist2', 'square', 'type1', 'type2'], axis = 1, inplace=True)

In [119]:
flats.rename(columns= {'dist3': 'district'}, inplace = True)

In [120]:
flats.isna().sum()

price              0
district           0
rooms              0
floor              0
year            7502
metro              0
total_area         0
living_area     5113
kitchen_area     246
floor_total      150
dtype: int64

In [121]:
flats.drop(['living_area', 'year'], axis = 1, inplace = True)

In [122]:
flats.to_csv('flats_cleaned.csv', index=False)